In [1]:
import time
import os
import numpy as np
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # disable infos
os.environ["TF_ENABLE_ONEDNN_OPTS"]="1" # make sure onednn is used on CPU
# tf.config.set_visible_devices([], 'GPU') # GPU improves c-times much

T=3.0
N=int(T*2*12)
Nsim = 10
# N=int(T*4*12) # no real difference
# Nsim = 5
r=0.0303
dtype=np.float32
seed=1
tf.random.set_seed(seed)

t=np.linspace(0,T,N*Nsim,endpoint=True,dtype=dtype).reshape((-1,1))
t=tf.constant(t)

from Commodities import Schwartz2Factor
'Salmon'
# mu, sigma1, sigma2, kappa, alpha, lambda, rho, delta0, P0
# salmonParam=[0.12, 0.23, 0.75, 2.6, 0.02, 0.01, 0.9, 0.57, 95] # down,down
salmonParam=[0.12, 0.23, 0.75, 2.6, 0.02, 0.2, 0.9, 0.57, 95] # down,up
# salmonParam=[0.12, 0.23, 0.75, 2.6, 0.02, 0.6, 0.9, 0.57, 95] # up,up

'Soy'
# mu, sigma1, sigma2, kappa, alpha, lambda, rho, delta0, P0
# soyParam=[0.15, 0.5, 0.4, 1.2, 0.06, 0.14, 0.44, 0.0, 1] # low vol
soyParam=[0.15, 1, 0.4, 1.2, 0.06, 0.14, 0.44, 0.0, 1] # medium vol
# soyParam=[0.15, 2, 0.4, 1.2, 0.06, 0.14, 0.44, 0.0, 1] # high vol

'Risk neutral dynamics'
salmonParam[0]=r
soyParam[0]=r

"Fish feeding 25% of production cost, disease 30%, harvest 10%. Total production cost = 50% of price = labor, smolt, ..."
salmonPrice=salmonParam[-1] #NOK/KG
productionCosts=salmonPrice*0.5
harvestingCosts=productionCosts*0.1 # roughly 10%
feedingCosts=productionCosts*0.25
biologicalCosts=productionCosts*0.3
initialSalmon=(salmonPrice-productionCosts)+feedingCosts+harvestingCosts+biologicalCosts #we add the costs to salmon price since they are respected in the model, other costs are fixed and thus removed
salmonParam[-1]=initialSalmon
print(f'Feeding costs {feedingCosts} and Harvesting costs {harvestingCosts}')


soy=Schwartz2Factor(soyParam,t,dtype=dtype)
salmon=Schwartz2Factor(salmonParam,t,dtype=dtype)


from Harvest import Harvest
hc = harvestingCosts
harvest = Harvest(hc)

from Growth import Bertalanffy
wInf=6
a=1.113
b=1.097
c=1.43
growth = Bertalanffy(t,wInf,a,b,c)

from Price import Price
price = Price(salmon)

from Feed import StochFeed,DetermFeed
cr=1.1
fc=feedingCosts
feed_s = StochFeed(fc,cr,r,t,soy)
feed_d = DetermFeed(fc,cr,r,t,soy)

from Mortality import ConstMortatlity,HostParasite,DetermHostParasite,Poisson,DetermPoisson
# n0=10000
# m=0.1
# mort = ConstMortatlity(t,n0,m) # not comparable to the other models, because of neglected treatment costs

params=[0.05,0.1,7.01,0.05]
beta=[0.0829,0.0281]
H0=10000.0
P0=10
treatmentCosts=0.015
tData=np.array([0.018868,0.037736,0.056604,0.075472,0.09434,0.11321,0.13208,0.15094,0.16981,0.18868,0.20755,0.22642,0.24528,0.26415,0.28302,0.30189,0.32075,0.33962,0.35849,0.37736,0.39623,0.41509,0.43396,0.45283,0.4717,0.49057,0.50943,0.5283,0.54717,0.56604,0.58491,0.60377,0.62264,0.64151,0.66038,0.67925,0.69811,0.71698,0.73585,0.75472,0.77358,0.79245,0.81132,0.83019,0.84906,0.86792,0.88679,0.90566,0.92453,0.9434,0.96226,0.98113,1,1.0189,1.0377,1.0566,1.0755,1.0943,1.1132,1.1321,1.1509,1.1698,1.1887,1.2075,1.2453,1.2642,1.283,1.3019,1.3208,1.3396,1.3585,1.3774,1.3962,1.4151,1.434,1.4528,1.4717,1.4906,1.5094,1.5283,1.5472,1.566,1.5849,1.6038,1.6226,1.6415,1.6604,1.6792,1.6981,1.717,1.7358,1.7547,1.7736
],dtype=np.float32).reshape((-1,))
dm=np.array([3.3027e-05,9.7183e-05,0.00026979,0.00070556,0.0017367,0.0040223,0.008768,0.018005,0.034877,0.063854,0.11076,0.18249,0.28649,0.42987,0.61849,0.85609,1.1437,1.4794,1.8584,2.2738,2.7168,3.1781,3.6482,4.118,4.5797,5.0265,5.4533,5.856,6.2323,6.5805,6.9003,7.1919,7.4562,7.6944,7.9081,8.0991,8.2691,8.42,8.5536,8.6715,8.7755,8.8669,8.9472,9.0177,9.0794,9.1334,9.1806,9.2218,9.2578,9.2893,9.3167,9.3405,9.3614,9.3795,9.3953,9.409,9.421,9.4314,9.4405,9.4483,9.4552,9.4612,9.4664,9.4748,9.4782,9.4812,9.4837,9.486,9.4879,9.4896,9.4911,9.4924,9.4935,9.4944,9.4953,9.496,9.4967,9.4972,9.4977,9.4981,9.4985,9.4988,9.4991,9.4993,9.4995,9.4997,9.4998,9.5,9.5001,9.5002,9.5003,9.5004
],dtype=np.float32).reshape((-1,))

# mort_s = HostParasite(t,params,beta,H0,P0,treatmentCosts=treatmentCosts)
# mort_d = DetermHostParasite(t,params,beta,H0,P0,treatmentCosts=treatmentCosts)

mort_s = Poisson(H0,params[0],t,tData[1:],dm,treatmentCosts=treatmentCosts)
mort_d = DetermPoisson(H0,params[0],t,tData[1:],dm,treatmentCosts=treatmentCosts)

from FishFarm import fishFarm
farm_ss = fishFarm(growth,feed_s,price,harvest,mort_s,stride=Nsim,seed=seed)
farm_sd = fishFarm(growth,feed_s,price,harvest,mort_d,stride=Nsim,seed=seed)
farm_ds = fishFarm(growth,feed_d,price,harvest,mort_s,stride=Nsim,seed=seed)
farm_dd = fishFarm(growth,feed_d,price,harvest,mort_d,stride=Nsim,seed=seed)

from OptimalStopping import Polynomial,DeepOptS,LSMC
batch_size=2**12 #need to fix it for simplicity
batches=20 # not so relevant since we make pathwise comparison, only relevant for value of opt stopping

Feeding costs 11.875 and Harvesting costs 4.75


c:\ProgramData\Anaconda3\envs\intelpython310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
farm_ss.seed(seed)
tf.random.set_seed(seed)
X_ss,V_ss,Vh_ss,ft_ss = farm_ss.generateFishFarm(batch_size*batches) # make sure to evaluate on same data and compiles all code for generating data
X_ss=np.array(X_ss)
V_ss=np.array(V_ss)
Vh_ss=np.array(Vh_ss)
ft_ss=np.array(ft_ss)
print(X_ss.shape)

(73, 81920, 5)


In [3]:
farm_sd.seed(seed)
tf.random.set_seed(seed)
X_sd,V_sd,Vh_sd,ft_sd = farm_sd.generateFishFarm(batch_size*batches) # make sure to evaluate on same data and compiles all code for generating data
X_sd=np.array(X_sd)
V_sd=np.array(V_sd)
Vh_sd=np.array(Vh_sd)
ft_sd=np.array(ft_sd)
print(X_sd.shape)

(73, 81920, 4)


In [4]:
farm_ds.seed(seed)
tf.random.set_seed(seed)
X_ds,V_ds,Vh_ds,ft_ds = farm_ds.generateFishFarm(batch_size*batches) # make sure to evaluate on same data and compiles all code for generating data
X_ds=np.array(X_ds)
V_ds=np.array(V_ds)
Vh_ds=np.array(Vh_ds)
ft_ds=np.array(ft_ds)
print(X_ds.shape)

(73, 81920, 3)


In [5]:
farm_dd.seed(seed)
tf.random.set_seed(seed)
X_dd,V_dd,Vh_dd,ft_dd = farm_dd.generateFishFarm(batch_size*batches) # make sure to evaluate on same data and compiles all code for generating data
X_dd=np.array(X_dd)
V_dd=np.array(V_dd)
Vh_dd=np.array(Vh_dd)
ft_dd=np.array(ft_dd)
print(X_dd.shape)

(73, 81920, 2)


In [6]:
batches=20
farm_ss.seed(seed+1)
tf.random.set_seed(seed+1)
optDeep_ss=DeepOptS(r,farm_ss.tCoarse,farm_ss.generateFishFarm,d=farm_ss.d,batch_size=batch_size)
optDeep_ss.train(batches)

tic=time.time()
tauDOS_ss,VtauDOS_ss=optDeep_ss.evaluate(X_ss,V_ss,Vh_ss,ft_ss)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tauDOS_ss)} with mean value {np.mean(VtauDOS_ss)} in {ctimeEval} s')


100%|██████████| 1505/1505 [00:36<00:00, 40.88epoch/s, loss=-2.39e+6, payoff=2.42e+6]


Mean stopping time 2.030149459838867 with mean value 2441395.5 in 0.7497267723083496 s


In [7]:
batches=20
farm_sd.seed(seed+1)
tf.random.set_seed(seed+1)
optDeep_sd=DeepOptS(r,farm_sd.tCoarse,farm_sd.generateFishFarm,d=farm_sd.d,batch_size=batch_size)
optDeep_sd.train(batches)

tic=time.time()
tauDOS_sd,VtauDOS_sd=optDeep_sd.evaluate(X_sd,V_sd,Vh_sd,ft_sd)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tauDOS_sd)} with mean value {np.mean(VtauDOS_sd)} in {ctimeEval} s')


100%|██████████| 1504/1504 [00:33<00:00, 44.59epoch/s, loss=-2.42e+6, payoff=2.45e+6]


Mean stopping time 2.0248379707336426 with mean value 2438669.5 in 0.6772334575653076 s


In [8]:
batches=20
farm_ds.seed(seed+1)
tf.random.set_seed(seed+1)
optDeep_ds=DeepOptS(r,farm_ds.tCoarse,farm_ds.generateFishFarm,d=farm_ds.d,batch_size=batch_size)
optDeep_ds.train(batches)

tic=time.time()
tauDOS_ds,VtauDOS_ds=optDeep_ds.evaluate(X_ds,V_ds,Vh_ds,ft_ds)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tauDOS_ds)} with mean value {np.mean(VtauDOS_ds)} in {ctimeEval} s')


100%|██████████| 1503/1503 [00:34<00:00, 43.74epoch/s, loss=-2.36e+6, payoff=2.39e+6]


Mean stopping time 2.039902687072754 with mean value 2376900.5 in 0.6265823841094971 s


In [9]:
batches=20
farm_dd.seed(seed+1)
tf.random.set_seed(seed+1)
optDeep_dd=DeepOptS(r,farm_dd.tCoarse,farm_dd.generateFishFarm,d=farm_dd.d,batch_size=batch_size)
optDeep_dd.train(batches)

tic=time.time()
tauDOS_dd,VtauDOS_dd=optDeep_dd.evaluate(X_dd,V_dd,Vh_dd,ft_dd)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tauDOS_dd)} with mean value {np.mean(VtauDOS_dd)} in {ctimeEval} s')


100%|██████████| 1502/1502 [00:32<00:00, 45.80epoch/s, loss=-2.36e+6, payoff=2.39e+6]


Mean stopping time 2.03869891166687 with mean value 2373908.5 in 0.7911098003387451 s


In [10]:
tComp,Vcomp = fishFarm.compareStoppingTimes(V_ss,[tauDOS_ss,tauDOS_sd,tauDOS_ds,tauDOS_dd],np.array(farm_ss.tCoarse))
for i in range(0,len(tComp)):
    print(f'Mean tau: {tComp[i]} with mean {Vcomp[i]} with ratio {Vcomp[0]/Vcomp[i]}')

Mean tau: [2.03040524] with mean 2441395.292510274 with ratio 1.0
Mean tau: [2.02508602] with mean 2440482.6178490906 with ratio 1.0003739730225931
Mean tau: [2.04017849] with mean 2375847.9316923143 with ratio 1.0275890388200353
Mean tau: [2.03897285] with mean 2374750.082989724 with ratio 1.0280640939852694


In [11]:
basis_ss = Polynomial(deg=2,dtype=dtype)
basis_sd = Polynomial(deg=2,dtype=dtype)
basis_ds = Polynomial(deg=2,dtype=dtype)
basis_dd = Polynomial(deg=2,dtype=dtype)
optLSMC_ss=LSMC(r,farm_ss.tCoarse,farm_ss.generateFishFarm,basis_ss,batch_size=batch_size)
optLSMC_sd=LSMC(r,farm_sd.tCoarse,farm_sd.generateFishFarm,basis_sd,batch_size=batch_size)
optLSMC_ds=LSMC(r,farm_ds.tCoarse,farm_ds.generateFishFarm,basis_ds,batch_size=batch_size)
optLSMC_dd=LSMC(r,farm_dd.tCoarse,farm_dd.generateFishFarm,basis_dd,batch_size=batch_size)

tic=time.time()
tau_ss,Vtau_ss=optLSMC_ss.evaluate(X_ss,V_ss,Vh_ss,ft_ss)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tau_ss)} with mean value {np.mean(Vtau_ss)} in {ctimeEval} s')

tic=time.time()
tau_sd,Vtau_sd=optLSMC_sd.evaluate(X_sd,V_sd,Vh_sd,ft_sd)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tau_sd)} with mean value {np.mean(Vtau_sd)} in {ctimeEval} s')

tic=time.time()
tau_ds,Vtau_ds=optLSMC_ds.evaluate(X_ds,V_ds,Vh_ds,ft_ds)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tau_ds)} with mean value {np.mean(Vtau_ds)} in {ctimeEval} s')

tic=time.time()
tau_dd,Vtau_dd=optLSMC_dd.evaluate(X_dd,V_dd,Vh_dd,ft_dd)
ctimeEval=time.time()-tic
print(f'Mean stopping time {np.mean(tau_dd)} with mean value {np.mean(Vtau_dd)} in {ctimeEval} s')

Mean stopping time 1.8990395069122314 with mean value 2342078.0 in 4.764180660247803 s
Mean stopping time 2.0472216606140137 with mean value 2443610.75 in 1.9006409645080566 s
Mean stopping time 1.9085391759872437 with mean value 2277907.0 in 1.6212143898010254 s
Mean stopping time 2.0668725967407227 with mean value 2387305.75 in 1.9039766788482666 s


In [12]:
tComp,Vcomp = fishFarm.compareStoppingTimes(V_ss,[tau_ss,tau_sd,tau_ds,tau_dd],np.array(farm_ss.tCoarse))
for i in range(0,len(tComp)):
    print(f'Mean tau: {tComp[i]} with mean {Vcomp[i]} with ratio {Vcomp[0]/Vcomp[i]}')

Mean tau: 1.8990395342851116 with mean 2342078.1821526526 with ratio 1.0
Mean tau: 2.047222028299802 with mean 2445511.2201789855 with ratio 0.9577049423560761
Mean tau: 1.9085391175533004 with mean 2276663.428049469 with ratio 1.0287327293517547
Mean tau: 2.0668726320873247 with mean 2388445.9379203795 with ratio 0.9805866421209017
